# Ref
https://github.com/shafitek/DeepChess-AI/blob/master/scripts/DeepChess.py

In [1]:
import pandas as pd
import numpy as np
import os 
import chess
import chess.engine
import chess.svg
import random
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from models.MiniDeepchess import SiameseNetwork, AutoEncoder
from tqdm import tqdm

In [2]:
class AutoEncoderChessDataset(Dataset):
    def __init__(self, whiteWonStates, whiteLostStates):
        """
        Dataset dùng để huấn luyện AutoEncoder từ các trạng thái thắng và thua.

        Args:
            whiteWonStates (np.ndarray): Trạng thái bàn cờ khi trắng thắng.
            whiteLostStates (np.ndarray): Trạng thái bàn cờ khi trắng thua (tức đen thắng).
        """
        sampleSize = min(len(whiteWonStates), len(whiteLostStates))
        whiteWonStates = whiteWonStates.copy()
        whiteLostStates = whiteLostStates.copy()
        np.random.shuffle(whiteWonStates)
        np.random.shuffle(whiteLostStates)
        whiteWonStates = whiteWonStates[:sampleSize]
        whiteLostStates = whiteLostStates[:sampleSize]

        # Gán label: trắng thắng → 1, trắng thua (đen thắng) → 0
        whiteWonLabels = np.ones((sampleSize, 1))
        whiteLostLabels = np.zeros((sampleSize, 1))

        self.data = np.concatenate((whiteWonStates, whiteLostStates), axis=0)
        self.labels = np.concatenate((whiteWonLabels, whiteLostLabels), axis=0)

        # Shuffle cùng lúc data và label
        perm = np.random.permutation(len(self.data))
        self.data = self.data[perm]
        self.labels = self.labels[perm]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        """
        Trả về trạng thái bàn cờ và nhãn dưới dạng tensor.

        Returns:
            x (Tensor): Trạng thái bàn cờ.
            y (Tensor): Nhãn (1 nếu trắng thắng, 0 nếu trắng thua).
        """
        x = torch.from_numpy(self.data[index]).float()
        y = torch.tensor(self.labels[index]).float().squeeze()  # squeeze để có dạng scalar
        return x, y


In [3]:
percentTrain = 0.8

whiteWin = np.load("./data/whiteWin.npy")
whiteLost = np.load("./data/blackWin.npy")

whiteWinTrain = whiteWin[:int(len(whiteWin) * percentTrain)]
whiteLostTrain = whiteLost[:int(len(whiteLost) * percentTrain)]
whiteWinTest = whiteWin[int(len(whiteWin) * percentTrain):]
whiteLostTest = whiteLost[int(len(whiteLost) * percentTrain):]

print(f"Train: {len(whiteWinTrain)} white wins, {len(whiteLostTrain)} black wins")
print(f"Test: {len(whiteWinTest)} white wins, {len(whiteLostTest)} black wins")

dataset_train = AutoEncoderChessDataset(
    whiteWonStates=whiteWinTrain,
    whiteLostStates=whiteLostTrain
)
dataloader_train = DataLoader(dataset_train, batch_size=512, shuffle=True)

dataset_test = AutoEncoderChessDataset(
    whiteWonStates=whiteWinTest,
    whiteLostStates=whiteLostTest
)
dataloader_test = DataLoader(dataset_test, batch_size=512, shuffle=True)

Train: 1600000 white wins, 1600000 black wins
Test: 400000 white wins, 400000 black wins


In [ ]:
# model = AutoEncoder(
#     layer=[773, 600, 400, 200, 100]
# )
# model_path = "./checkpoints/AutoEncoder.pth"
# if os.path.exists(model_path):
#     print("Loading AutoEncoder model from checkpoint...")
#     model.load_state_dict(torch.load(model_path, map_location='cpu'))
# model.eval()

Loading AutoEncoder model from checkpoint...


RuntimeError: Error(s) in loading state_dict for AutoEncoder:
	Missing key(s) in state_dict: "encoder_layers.0.0.weight", "encoder_layers.0.0.bias", "encoder_layers.0.1.weight", "encoder_layers.0.1.bias", "encoder_layers.0.1.running_mean", "encoder_layers.0.1.running_var", "encoder_layers.1.0.weight", "encoder_layers.1.0.bias", "encoder_layers.1.1.weight", "encoder_layers.1.1.bias", "encoder_layers.1.1.running_mean", "encoder_layers.1.1.running_var", "encoder_layers.2.0.weight", "encoder_layers.2.0.bias", "encoder_layers.2.1.weight", "encoder_layers.2.1.bias", "encoder_layers.2.1.running_mean", "encoder_layers.2.1.running_var", "encoder_layers.3.0.weight", "encoder_layers.3.0.bias", "encoder_layers.3.1.weight", "encoder_layers.3.1.bias", "encoder_layers.3.1.running_mean", "encoder_layers.3.1.running_var", "decoder_layers.0.0.weight", "decoder_layers.0.0.bias", "decoder_layers.0.1.weight", "decoder_layers.0.1.bias", "decoder_layers.0.1.running_mean", "decoder_layers.0.1.running_var", "d

In [ ]:
def loss_function(recon_x, x):
    return F.binary_cross_entropy(recon_x, x, reduction='sum')

def mse_loss_function(recon_x, x):
    return F.mse_loss(recon_x, x, reduction='sum')

In [ ]:
# def train(model, dataloader, optimizer, mse_loss_function, device, model_path):
#     model.train()
#     running_loss = 0.0
    
#     for i, (x, _) in enumerate(tqdm(dataloader)):
#         x = x.to(device)

#         optimizer.zero_grad()

#         # Forward pass
#         recon_batch, _ = model(x)
#         loss = mse_loss_function(recon_batch, x)

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     train_loss = running_loss / len(dataloader)
#     return train_loss


# @torch.no_grad()
# def test(model, dataloader, mse_loss_function, device):
#     model.eval()
#     total_loss = 0.0

#     for x, _ in dataloader:
#         x = x.to(device)

#         recon_batch, _ = model(x)
#         loss = mse_loss_function(recon_batch, x)
#         total_loss += loss.item()

#     avg_loss = total_loss / len(dataloader)
#     print(f"Test Loss: {avg_loss:.6f}")
#     return avg_loss

In [ ]:
# epochs = 1000
# patience = 5
# best_loss = float('inf')
# patience = 5

# for epoch in range(epochs):
#     train_loss = train(model, dataloader_train, optimizer, mse_loss_function, device, model_path)
#     test_loss = test(model, dataloader_test, mse_loss_function, device)
    
#     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}")
    
#     if test_loss < best_loss:
#         best_loss = test_loss
#         patience_counter = 0
#         torch.save(model.state_dict(), model_path)
#         print("✅ Saved best model")
#     else:
#         patience_counter += 1
#         print(f"⏸️ No improvement. Patience counter: {patience_counter}/{patience}")
#         if patience_counter >= patience:
#             print("🛑 Early stopping triggered")
#             break

 71%|███████   | 17637/25000 [02:31<00:59, 124.66it/s]

In [ ]:
model = AutoEncoder(
    layer =[773, 600, 400, 200, 100]
)
def layerwise_pretrain(model, epochs=30, lr=1e-3, patience=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    pretrained_weights = []

    for i in range(len(model.layer) - 1):
        print(f"Pretraining layer {i}: {model.layer[i]} → {model.layer[i+1]}")

        # Shallow Autoencoder
        encoder = model.encoder_layers[i][0]  # Linear layer
        decoder = nn.Linear(model.layer[i + 1], model.layer[i]).to(device)

        optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=lr)
        criterion = nn.MSELoss()

        for epoch in range(epochs):
            total_loss = 0
            for x_batch, _ in dataloader_train:
                x_batch = x_batch.to(device)
                optimizer.zero_grad()
                z = model.encoder_layers[i](x_batch)
                recon = decoder(z)
                loss = criterion(recon, x_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item() * x_batch.size(0)
            epoch_loss = total_loss / len(dataloader_train.dataset)

            if epoch_loss < best_loss - 1e-4:  # Some delta
                best_loss = epoch_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"  Early stopping at epoch {epoch+1}")
                    break
        # Save decoder for optional reconstruction later
        pretrained_weights.append((encoder.weight.data.clone(), encoder.bias.data.clone()))

        # Update current input to next layer
        with torch.no_grad():
            current_input = model.encoder_layers[i](current_input)

    return pretrained_weights

layerwise_pretrain(model, epochs=30, batch_size=128, lr=1e-2)

Pretraining layer 0: 773 → 600


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def layerwise_pretrain(model, trainX, epochs=30, lr=1e-3, patience=5, batch_size=128):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Initialize dataloader
    dataset = AutoEncoderChessDataset(trainX[0], trainX[1])
    dataloader_train = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    pretrained_weights = []

    for i in range(len(model.layer) - 1):
        print(f"Pretraining layer {i}: {model.layer[i]} → {model.layer[i+1]}")

        # Initialize best_loss and epochs_no_improve for early stopping
        best_loss = float('inf')
        epochs_no_improve = 0

        # Shallow Autoencoder
        encoder = model.encoder_layers[i][0]  # nn.Linear
        decoder = nn.Linear(model.layer[i + 1], model.layer[i]).to(device)

        optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=lr)
        criterion = nn.MSELoss()

        for epoch in range(epochs):
            total_loss = 0
            for x_batch, _ in dataloader_train:
                x_batch = x_batch.to(device)
                optimizer.zero_grad()
                z = model.encoder_layers[i](x_batch)
                recon = decoder(z)
                loss = criterion(recon, x_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item() * x_batch.size(0)

            epoch_loss = total_loss / len(dataloader_train.dataset)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.6f}")

            # Early stopping logic
            if epoch_loss < best_loss - 1e-4:  # Some delta
                best_loss = epoch_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"  Early stopping at epoch {epoch+1}")
                    break

        # Save decoder weights
        pretrained_weights.append((encoder.weight.data.clone(), encoder.bias.data.clone()))

        # Feedforward output for next layer
        with torch.no_grad():
            current_input = model.encoder_layers[i](current_input)

    return pretrained_weights

# Example usage
percentTrain = 0.8

whiteWin = np.load("./data/whiteWin.npy")
whiteLost = np.load("./data/blackWin.npy")

whiteWinTrain = whiteWin[:int(len(whiteWin) * percentTrain)]
whiteLostTrain = whiteLost[:int(len(whiteLost) * percentTrain)]
whiteWinTest = whiteWin[int(len(whiteWin) * percentTrain):]
whiteLostTest = whiteLost[int(len(whiteLost) * percentTrain):]

print(f"Train: {len(whiteWinTrain)} white wins, {len(whiteLostTrain)} black wins")
print(f"Test: {len(whiteWinTest)} white wins, {len(whiteLostTest)} black wins")

# Prepare training data
dataset_train = AutoEncoderChessDataset(
    whiteWonStates=whiteWinTrain,
    whiteLostStates=whiteLostTrain
)
dataloader_train = DataLoader(dataset_train, batch_size=512, shuffle=True)

# Pretrain with early stopping
model = AutoEncoder(layer=[773, 600, 400, 200, 100])
layerwise_pretrain(model, (whiteWinTrain, whiteLostTrain), epochs=30, batch_size=128, lr=1e-2)